## Sandbox

Un  `.ipynb` pour tester des idées :)

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()
using Graphs
using MetaGraphsNext
using JuMP
using Gurobi

ENV["GRB_LICENSE_FILE"] = "gurobi.lic"
include(joinpath("src", "utils.jl"));

In [ ]:
M = Model(Gurobi.Optimizer)

In [ ]:
infos = open(
    readInstance,
    joinpath("data", "20_USA-road-d.BAY.gr"),
);

In [ ]:
graph = graphFromData(infos);

In [ ]:
ResultWrapper = NamedTuple{
    (:primal_status, :dual_status, :term_status, :obj_value, :bound, :a),
    Tuple{
        ResultStatusCode,
        ResultStatusCode,
        TerminationStatusCode,
        Float64,
        Float64,
        Dict{Tuple{Int64, Int64}, Float64},
    }
}

function dualSolve(
    graph::MetaGraph;
    timelimit::Float64=time() + 3600.0,
)::ResultWrapper
    s::Int64 = graph[].s
    t::Int64 = graph[].t
    m = Model(Gurobi.Optimizer)
    set_silent(m)
    function flow_creation(i::Int64)::Int64
        return Int64(graph[i].is_s) - Int64(graph[i].is_t)
    end
    @variable(m, a[edge_labels(graph)], Bin)
    # Conservation du flot :
    @constraint(
        m,
        flow_cons[i in labels(graph)],
        (
            sum(a[(i, j)] for j in outneighbor_labels(graph, i))
            -
            sum(a[(j, i)] for j in inneighbor_labels(graph, i))
            == flow_creation(i)
        )
    )
    @variable(m, lambda_d >= 0)
    @variable(m, mu_d[edge_labels(graph)] >= 0)
    # Contrainte de (D_d)
    @constraint(
        m,
        d_dual_cons[(i, j) in edge_labels(graph)],
        lambda_d + mu_d[(i, j)] >= graph[i, j].d * a[(i, j)],
    )
    @variable(m, lambda_p >= 0)
    @variable(m, mu_p[labels(graph)] >= 0)
    # Contrainte de (D_p)
    @constraint(
        m,
        p_dual_cons[i in labels(graph)],
        lambda_p + mu_p[i]
        >=
        graph[i].ph * (graph[i].is_t + sum(a[(i, j)] for j in outneighbor_labels(graph, i))),
    )
    # Contrainte correspondant a l'objectif de (D_p)
    @constraint(
        m,
        p_dual_obj,
        (
            graph[t].p
            +
            sum(graph[i].p * a[(i, j)] for i in labels(graph) for j in outneighbor_labels(graph, i))
            +
            2 * sum(mu_p) + lambda_p * graph[].d2
            <= graph[].big_s
        )
    )
    # Objectif
    @objective(
        m,
        Min,
        graph[].d1 * lambda_d
        + sum(
            graph[i, j].d * a[(i, j)] 
            +
            graph[i, j].big_d * mu_d[(i, j)]
            for (i, j) in edge_labels(graph)
        )
    )
    set_attribute(m, "TimeLimit", timelimit - time())
    optimize!(m)
    return (
        primal_status=primal_status(m),
        dual_status=dual_status(m),
        term_status=termination_status(m),
        obj_value=objective_value(m),
        bound=objective_bound(m),
        a=Dict{Tuple{Int64, Int64}, Float64}((i, j) => value(a[(i, j)]) for (i, j) in edge_labels(graph)),
    )
end

In [ ]:
function mkPath(a::Dict{Tuple{Int64, Int64}, Float64})::Vector{Int64}
    building_dict::Dict{Int64, Int64} = Dict{Int64, Int64}()
    for ((i, j), val) in pairs(a)
        int_val::Int64 = Int64(val)
        floor(val) == val && int_val in [0, 1] ? nothing : error("a is not integer feasible.")
        if int_val == 1
            haskey(building_dict, i) ? error("Two arcs coming from the same node.") : building_dict[i] = j
        end
    end
    current_i::Int64 = only(setdiff(keys(building_dict), values(building_dict)))
    path::Vector{Int64} = [current_i]
    while haskey(building_dict, current_i)
        current_i = building_dict[current_i]
        push!(path, current_i)
    end
    return path
end

In [ ]:
res = dualSolve(graph; timelimit=time() + 60.0)
mkPath(res.a)